In [50]:
import numpy as np
from collections import defaultdict
from tic_tac_toe import TicTacToe, plot_test_game

In [47]:
env = TicTacToe(n_rows=4, n_cols=4, n_win=3)
# plot_test_game(env, None, None, random_crosses=True, random_naughts=True, verbose=True, fontx=60)

In [ ]:
class RandomPolicy:
    def __init__(self):
        pass
    
    def get_action(self, state):
        _, empty_spaces, _ = state
        return np.random.randint(len(empty_spaces))

In [48]:
env.getHash()

'1111111111111111'

In [51]:
Q = defaultdict(lambda: np.zeros(len(empty_spaces)))

In [53]:
Q['1111111111111111']

NameError: name 'empty_spaces' is not defined

In [ ]:
class EpsGreedyPolicy:
    def __init__(self):
        self.Q = {}

    def select_action(self, state, player: int = None, eps: float = 0.0):
        board_hash, empty_spaces, _ = state

        if random.random() >= eps and board_hash in self.Q:
            return np.argmax(self.Q[board_hash])

        if board_hash not in self.Q:
            self.Q[board_hash] = np.zeros(len(empty_spaces))

        return np.random.randint(len(empty_spaces))

In [36]:
env.printBoard()

╭───┬───┬───┬───╮
│ o │ x │ x │ x │ 
├───┼───┼───┼───┤
│ o │   │ x │   │ 
├───┼───┼───┼───┤
│   │   │   │   │ 
├───┼───┼───┼───┤
│   │   │ o │   │ 
╰───┴───┴───┴───╯
